In [2]:
import pandas as pd

In [6]:
# Задание 1**
# Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
# - оценка 2 и меньше - низкий рейтинг
# - оценка 4 и меньше - средний рейтинг
# - оценка 4.5 и 5 - высокий рейтинг

# Результат классификации запишите в столбец class

data_mv = pd.read_csv('ratings.csv')

def rating_check(rating):
    if rating <= 2.0 and rating > 0.0:
        return 'low'
    elif rating <= 4.0 and rating > 2.0:
        return 'average'
    elif rating <= 5.0 and rating > 4.0:
        return 'high'
    else:
        return 'undefined'

data_mv['class'] = data_mv['rating'].apply(rating_check)

data_mv.head(10)


,userId,movieId,rating,timestamp,class
0,1,296,5.0,1147880044,high
1,1,306,3.5,1147868817,average
2,1,307,5.0,1147868828,high
3,1,665,5.0,1147878820,high
4,1,899,3.5,1147868510,average
5,1,1088,4.0,1147868495,average
6,1,1175,3.5,1147868826,average
7,1,1217,3.5,1147878326,average
8,1,1237,5.0,1147868839,high
9,1,1250,4.0,1147868414,average


In [13]:
# Задание 2
# Используем файл keywords.csv.

# Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую
# принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона,
# то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города,
# то ставим ‘undefined’.

# Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

def check_geo(keywords):
    for geo in geo_data:
        for city in geo_data.get(geo):
            if city in keywords:
                return geo
    return 'undefined'

keywords = pd.read_csv('keywords.csv')
keywords['geo'] = keywords['keyword'].apply(check_geo)

geo_keywords = keywords[ (keywords['geo'] != 'undefined') ]
geo_keywords.head()


,keyword,shows,geo
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


In [99]:
# Задание 3 (бонусное)
# Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”.
# Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов
# из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens).
# Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

# При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

# 1. В переменную years запишите список из всех годов с 1950 по 2010.

# 2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска.
# Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
# - для каждой строки пройдите по всем годам списка years
# - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
# - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

# 3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

# 4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат
# по убыванию рейтинга

# Важно! При копировании юпитер ноутбука изменяет кавычки ’ на ` , что может способствовать ошибки в коде.
# В данной ситуации необходимо их просто заменить.

# Задаем список лет (я расширил его, так как в файле с данными диапазон шире заданного)
years_list = range(1901,2021)


# Функция определения года по id фильма - берем название из справочника фильмов
def get_movie_year(movieId):
    movie_title = data_movies[data_movies.movieId == movieId]['title'].values[0]
    for year in years_list:
        if str(year) in movie_title:
            return year
    return 1900

# Считываем файлы рейтингов и фильмов
data_ratings = pd.read_csv('ratings.csv')
data_movies = pd.read_csv('movies.csv')

# Проставляем год для всех фильмов из файла
# Делаем это, чтобы избежать лишних вычислений в файле с рейтингами
# data_movies['year'] = data_movies['title'].apply(get_movie_year)

# Считаем средний рейтинг по фильму, группируя по movieId
data_ratings = data_ratings.groupby('movieId').mean()[['rating']].reset_index()

# Для каждого id фильма определяем год - так меньше вычислений, поскольку movieId после группировки уникальный
data_ratings['year'] = data_ratings['movieId'].apply(get_movie_year)

# Считаем средний рейтинг по году, группируя по полю 'year'
data_ratings = data_ratings.groupby('year').mean()[['rating']].sort_values('rating', ascending=False)
data_ratings.head(50)


,rating
year,
1917,4.500000
1921,4.100000
1934,4.051190
1926,4.000000
1908,4.000000
1930,3.995000
1944,3.977261
1935,3.850871
1971,3.811243
